# QZFREG

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Tue Oct 26 15:49:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# ---------------------------------------------------------------------------------------------------

In [2]:
import os
import sys
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
import time
import glob

import torch
from torch.nn import functional as F
import torch.optim as optim
import torchvision
from torchvision.utils import save_image
import torch.utils.data as data_utils
import torch.distributions as dist


from sklearn.decomposition import PCA
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import random
from tqdm import tqdm

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3227691861626684629, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 13283098624
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9168655510130383982
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [3]:
if not os.path.isdir('drive'):
  drive.mount('drive')
else:
  print('drive already mounted')

base_path = os.path.join('drive', 'My Drive', 'Colab Notebooks', 'GEN')
if not os.path.isdir(base_path):
  os.makedirs(base_path)

Mounted at drive


In [4]:
PATH = os.path.join("drive", "My Drive", "Colab Notebooks", "GEN") 

sys.path.append(PATH+'/biomatsim')
from dataset.data_loader_synthetic import ToyCell, ToyCellpair
from model_vae_synthetic import VAE
from model_qzfreg_synthetic import QZFREG
from model_main_synthetic import FDVAE, REGVAE, Discriminator
from diva.pixel_cnn_utils import sample

print(torch.cuda.is_available())

True


In [5]:
# https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary
##########################
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
##########################

### Load data

In [6]:
# get cell data
dataset_name = 'dataset2_128_fixloc_1scale'

if not os.path.isdir('dataset2_128_fixloc_1scale'): 
  start_time = time.time()
  if not os.path.isfile('dataset2_128_fixloc_1scale.zip'): 
    ! wget -O dataset2_128_fixloc_1scale.zip "https://surfdrive.surf.nl/files/index.php/s/VuU2UKuHlBpvxgh/download"
  ! unzip -q dataset2_128_fixloc_1scale.zip -d .
  print("Unzipped.")
  print("Elapsed time: {} seconds.".format(time.time()-start_time))
else:
  print('Found folder dataset2_128_fixloc_1scale')

--2021-10-26 15:51:15--  https://surfdrive.surf.nl/files/index.php/s/VuU2UKuHlBpvxgh/download
Resolving surfdrive.surf.nl (surfdrive.surf.nl)... 145.100.27.67, 2001:610:108:203b:0:a11:da7a:5afe
Connecting to surfdrive.surf.nl (surfdrive.surf.nl)|145.100.27.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1280707500 (1.2G) [application/zip]
Saving to: ‘dataset2_128_fixloc_1scale.zip’

dataset2_128_fixloc 100%[===================>]   1.19G  11.8MB/s    in 1m 45s  

2021-10-26 15:53:02 (11.6 MB/s) - ‘dataset2_128_fixloc_1scale.zip’ saved [1280707500/1280707500]

Unzipped.
Elapsed time: 116.56779408454895 seconds.


### Train

In [7]:
def run_epoch_qzf(feature_index, data_loader, model, optimizer, train=True):
    #model.train()
    avg_loss = 0
    reg_qzf_loss = 0
    reg_pzf_loss = 0
    kl_zf = 0
    kl_zf_full = 0

    for batch_idx, (x, fs) in tqdm(enumerate(data_loader)):
        # To device
        x = x.to(device)
        for k in range(len(fs)):
            fs[k] = fs[k].to(device)
            # fs[k].shape = (batch_size, 1) if the feature is continuous; 
                        #   (batch_size, N) if the feature is categorical with N categories (one-hot)

        if train: 
          model.train()
          optimizer.zero_grad()
          loss, MSE, MSE2, KL_zf, KL_zf_full = model.loss_function(x, fs[feature_index]) 
          loss.backward()
          optimizer.step()
        else:
          with torch.no_grad():
            loss, MSE, MSE2, KL_zf, KL_zf_full = model.loss_function(x, fs[feature_index])

        avg_loss += loss.item()
        reg_qzf_loss += MSE.item()
        reg_pzf_loss += MSE2.item()
        kl_zf += KL_zf.item()
        kl_zf_full += KL_zf_full.item()

    avg_loss /= len(data_loader.dataset)
    reg_qzf_loss /= len(data_loader.dataset)
    reg_pzf_loss /= len(data_loader.dataset)
    kl_zf /= len(data_loader.dataset)
    kl_zf_full /= len(data_loader.dataset)

    return avg_loss, reg_qzf_loss, reg_pzf_loss, kl_zf, kl_zf_full

Setting the parameters

In [8]:
#### QZFREG
args = {
    'no_cuda': False, 
    'seed': 2, 
    'batch_size': 50,
    'epochs': 500,
    'lr': 0.01,

    # data info
    'datapath': 'dataset2_128_fixloc_1scale/', 
    'data_info_filename': 'dataset2_128_fixloc_1scale.csv', 
    'features_names': ['roundness', 'elongation', 'nucleus_size'],  #'rotation_angle'],
    'feature_index': 0,  ################# which feature to predict [0, 1, 2, 3]

    'f_dim': 1,
    'zf_dim': 2,
    'qf_act_bool': False,

    # loss multipliers
    'beta_f': 10,
    'beta_f_full': 1,
    'aux_loss_multiplier_qzf': 100000,
    'aux_loss_multiplier_pzf': 0,
    'prior_scale_train': True,

    # warm-up
    'w': 10, #5,  #100,  # 'number of epochs for warm-up. Set to 0 to turn warmup off.' (DIVA paper: https://github.com/AMLab-Amsterdam/DIVA)
    'max_beta': 1.,  # 'max beta for warm-up'
    'min_beta': 0.,

    'outpath': PATH+'/model_output_synthetic/qzfreg',  #'./'    
}

args = dotdict(args)
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")
kwargs = {'num_workers': 0, #1,   # https://github.com/pytorch/pytorch/issues/5301
          'pin_memory': False} if args.cuda else {}

In [9]:
##### Model name
model_name = 'QZFREG_seed_' + str(args.seed) + '_fi0_10_1_100k_0_scaletrain_v1'

model_path = args.outpath + '/' + model_name
print(args.outpath)
print(model_name)
print(model_path)

# Set seed
torch.manual_seed(args.seed)
torch.backends.cudnn.benchmark = False
np.random.seed(args.seed)

drive/My Drive/Colab Notebooks/GEN/model_output_synthetic/qzfreg
QZFREG_seed_2_fi0_10_1_100k_0_scaletrain_v1
drive/My Drive/Colab Notebooks/GEN/model_output_synthetic/qzfreg/QZFREG_seed_2_fi0_10_1_100k_0_scaletrain_v1


# ----------------------

Data loader

In [10]:
data_info_table = pd.read_csv(args.datapath + args.data_info_filename, index_col=0)
data_info_table['filename'] = data_info_table['idx'].apply(lambda x: str(x).zfill(7) + '.png')
data_info_table.drop('idx', axis=1, inplace=True)

###
np.random.seed(1)
img_list_val = list(np.random.choice(data_info_table['filename'].values, int(round(data_info_table.shape[0] * 0.2)), replace=False))
img_list_train = [img for img in data_info_table['filename'].values if img not in img_list_val]

print(len(img_list_train), len(img_list_val))

40000 10000


In [11]:
# import glob
# c1 = "00[0-2]***"
# c2 = "0030000"
# file_list = sorted(list(set(glob.glob(args.datapath + c1 + ".png")).union(
#                         set(glob.glob(args.datapath + c2 + ".png")))), reverse=True)

train_data = ToyCell(path=args.datapath, 
                     data_info_filename=args.data_info_filename, 
                     features_names=args.features_names,
                     imgsize=(128, 128),
                     #condition="00[0]*",
                     img_list=img_list_train,
                     scaler=True
                     )
train_loader = data_utils.DataLoader(train_data, batch_size=args.batch_size, shuffle=True, **kwargs)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
100%|██████████| 40000/40000 [00:50<00:00, 796.82it/s]


In [12]:
val_data = ToyCell(path=args.datapath, 
                   data_info_filename=args.data_info_filename, 
                   features_names=args.features_names,
                   imgsize=(128, 128),
                   #condition="00[0]*",
                   img_list=img_list_val,
                   scaler=True
                   )
val_loader = data_utils.DataLoader(val_data, batch_size=args.batch_size, shuffle=True, **kwargs)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
100%|██████████| 10000/10000 [00:12<00:00, 823.02it/s]


In [13]:
# print(train_data.ndf.shape)
# train_data.ndf.head()

In [14]:
#train_data.ndf['roundness'].hist(bins=30);

# ----------------------

Setup the model

In [15]:
################################# QZFREG
model = QZFREG(args).to(device)
model.cuda()

QZFREG(
  (pzf): pzf(
    (fc1s): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc21s): Sequential(
      (0): Linear(in_features=1, out_features=2, bias=True)
    )
    (fc22s): Sequential(
      (0): Linear(in_features=1, out_features=2, bias=True)
      (1): Softplus(beta=1, threshold=20)
    )
  )
  (qzf): Encoder(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rn1): IdResidualConvBlockBNResize(
      (nonlin): LeakyReLU(negative_slope=0.01)
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [ ]:
############ MORE TRAINING
# model = torch.load(model_path + '/' + model_name + '_epoch_10' + '.model')#, map_location=torch.device('cpu'))
# model = model.to(device)

In [ ]:
############ INITIALIZE FROM ANOTHER MODEL
# base_model_name = 'QZFREG_seed_1_fi0_10_1_100k_0_scaletrain_v0'
# base_model_path = 'drive/My Drive/Colab Notebooks/GEN/model_output_synthetic/qzfreg/QZFREG_seed_1_fi0_10_1_100k_0_scaletrain_v0'
# model = torch.load(base_model_path + '/' + base_model_name + '_epoch_20' + '.model')#, map_location=torch.device('cpu'))
# model = model.to(device)

# ----------------------

Setup the optimizer

In [16]:
print(args.lr)

0.01


In [17]:
# args.lr = 0.001

In [18]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr)

best_loss = 1e+8
best_y_acc = 0.

early_stopping_counter = 1
max_early_stopping = 100

# ----------------------

In [19]:
# check the arguments
args

{'aux_loss_multiplier_pzf': 0,
 'aux_loss_multiplier_qzf': 100000,
 'batch_size': 50,
 'beta_f': 10,
 'beta_f_full': 1,
 'cuda': True,
 'data_info_filename': 'dataset2_128_fixloc_1scale.csv',
 'datapath': 'dataset2_128_fixloc_1scale/',
 'epochs': 500,
 'f_dim': 1,
 'feature_index': 0,
 'features_names': ['roundness', 'elongation', 'nucleus_size'],
 'lr': 0.01,
 'max_beta': 1.0,
 'min_beta': 0.0,
 'no_cuda': False,
 'outpath': 'drive/My Drive/Colab Notebooks/GEN/model_output_synthetic/qzfreg',
 'prior_scale_train': True,
 'qf_act_bool': False,
 'seed': 2,
 'w': 10,
 'zf_dim': 2}

In [20]:
model_name, model_path

('QZFREG_seed_2_fi0_10_1_100k_0_scaletrain_v1',
 'drive/My Drive/Colab Notebooks/GEN/model_output_synthetic/qzfreg/QZFREG_seed_2_fi0_10_1_100k_0_scaletrain_v1')

In [19]:
# run only once when a new model is initialized
os.mkdir(model_path)

import json

with open(model_path + '/' + model_name + '.json', 'w') as configfile:
    json.dump(args, configfile, indent=2)

In [21]:
# check the number of parameters
def nparams(module):
    return sum(p.numel() for p in module.parameters() if p.requires_grad)

nparams(model)

557555

# -------------------------

TRAINING QZFREG

In [22]:
# check
# model.beta_f, model.beta_f_full, args.beta_f, args.beta_f_full,\
# model.aux_loss_multiplier_qzf, model.aux_loss_multiplier_pzf, args.aux_loss_multiplier_qzf, args.aux_loss_multiplier_pzf

In [23]:
import copy
args_COPY = copy.deepcopy(dict(args))

In [ ]:
######## Training loop QZFREG
print('\nStart training:', args)
for epoch in range(1, args.epochs + 1):

    ######################################
    ### WARMUP (increase beta gradually).
    ### Formula: min([final_value, curent_val + (final value - current_val) * (epoch * 1. - epochs_completed) / (args_COPY['w'] - epochs_completed)])
    # model.beta_f = min([args_COPY['beta_f'], 0 + (args_COPY['beta_f'] - 0) * (epoch * 1. - 0) / (args_COPY['w'] - 0)])
    # print('BETAS: ',  model.beta_f, model.beta_f_full, args_COPY['beta_f'], args_COPY['beta_f_full'])
    ######################################

    # Train
    train_loss, reg_qzf_loss, reg_pzf_loss, kl_zf, kl_zf_full = run_epoch_qzf(args.feature_index, train_loader, model, optimizer, train=True)

    # logging train scores
    str_print = f" {epoch} EPOCH: avg loss {train_loss}"
    str_print += f" avg reg_qzf_loss {reg_qzf_loss}"
    str_print += f" avg reg_pzf_loss {reg_pzf_loss}"
    str_print += f" avg kl_zf {kl_zf}"
    str_print += f" avg kl_zf_full {kl_zf_full}"
    print(str_print) 

    # Val
    val_loss, val_reg_qzf_loss, val_reg_pzf_loss, val_kl_zf, val_kl_zf_full = run_epoch_qzf(args.feature_index, val_loader, model, optimizer, train=False)

    # logging val scores
    str_print = f" {epoch} EPOCH: VAL avg loss {val_loss}"
    str_print += f" avg reg_qzf_loss {val_reg_qzf_loss}"
    str_print += f" avg reg_pzf_loss {val_reg_pzf_loss}"
    str_print += f" avg kl_zf {val_kl_zf}"
    str_print += f" avg kl_zf_full {val_kl_zf_full}"
    print(str_print) 

    if val_loss < best_loss:
        early_stopping_counter = 1

        best_loss =  val_loss

        print("saving model: ", model_name + f'_epoch_{epoch}' + '.model')
        torch.save(model, model_path + '/' + model_name + f'_epoch_{epoch}' + '.model')
    else:
        #######
        # if epoch >= 1 and epoch % 10 == 0:
        #     print("RESERVE saving model: ", model_name + f'_epoch_{epoch}' + '.model')
        #     torch.save(model, model_path + '/' + model_name + f'_epoch_{epoch}' + '.model')
        #######   
        early_stopping_counter += 1
        if early_stopping_counter == max_early_stopping:
            break

In [ ]:
# 800it [01:51,  7.16it/s]
#  1 EPOCH: avg loss 323.34898185424805 avg reg_qzf_loss 0.0029067469115951097 avg reg_pzf_loss 0.006460839280905202 avg kl_zf 3.1678877178192137 avg kl_zf_full 0.9954157078683377
# 200it [00:06, 32.58it/s]
#  1 EPOCH: VAL avg loss 46.99279559326172 avg reg_qzf_loss 0.0003218131815548986 avg reg_pzf_loss 0.0005997189030051231 avg kl_zf 1.3025865314483642 avg kl_zf_full 1.7856119979858398
# saving model:  QZFREG_seed_2_fi0_10_1_100k_0_scaletrain_v1_epoch_1.model